In [ ]:
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from flexmeasures.data.models.planning.linear_optimization import device_scheduler
from flexmeasures.data.models.planning.utils import initialize_series, initialize_df

from flexmeasures.data.models.planning import (
    FlowCommitment,
    Scheduler,
    SchedulerOutputType,
    StockCommitment,
)

from flexmeasures.data.models.planning.storage import StorageScheduler

from pyomo.environ import value

TOLERANCE = 0.00001

from flexmeasures.app import create

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
app = create()

## Simultaneous scheduling test for 1 day

In [ ]:
start = pd.Timestamp("2020-01-01T00:00:00")
end = pd.Timestamp("2020-01-02T00:00:00")
resolution = timedelta(hours=1)
soc_at_start = [3, 2, 5, 4] * 4 
soc_max = [11] * 16
soc_min = [0] * 16

start_datetime = ["2020-01-01 09:00:00"] * 16
target_datetime = ["2020-01-01 15:00:00", "2020-01-01 17:00:00", "2020-01-01 15:00:00", "2020-01-01 16:00:00"] * 4
target_value = [11] * 16

market_prices = [
    0.8598, 1.4613, 2430.3887, 3000.1779, 18.6619, 369.3274, 169.8719, 174.2279, 174.2279, 174.2279,
    175.4258, 1.5697, 174.2763, 174.2279, 175.2564, 202.6992, 218.4413, 229.9242, 295.1069, 240.7174,
    249.2479, 238.2732, 229.8395, 216.5779
]

def initialize_combined_constraints(num_devices, soc_at_start, soc_max, soc_min, target_datetime, target_value, start_datetime):
    device_constraints = []
    for i in range(num_devices):
        constraints = initialize_df(StorageScheduler.COLUMNS, start, end, resolution)

        start_time = pd.Timestamp(start_datetime[i]) - timedelta(hours=1)

        constraints["max"] = soc_max[i] - soc_at_start[i]
        constraints["min"] = soc_min[i] - soc_at_start[i]
        constraints["derivative max"] = 11
        constraints["derivative min"] = 0
        constraints["min"][target_datetime[i]] = target_value[i] - soc_at_start[i]
        constraints.loc[:start_time, ["max", "min", "derivative max", "derivative min"]] = 0
        device_constraints.append(constraints)
    return device_constraints

def initialize_combined_commitments(num_devices):
    commitments = []
    for _ in range(num_devices):
        commitment = initialize_df(
            ["quantity", "downwards deviation price", "upwards deviation price", "group"],
            start, end, resolution
        )
        commitment["quantity"] = 0
        commitment["downwards deviation price"] = market_prices
        commitment["upwards deviation price"] = market_prices
        commitment["group"] = list(range(len(commitment)))
        commitments.append(commitment)
    return commitments

def run_simultaneous_scheduler():
    with app.app_context():
        num_devices = len(soc_at_start)
        device_constraints = initialize_combined_constraints(
            num_devices, soc_at_start, soc_max, soc_min, target_datetime, target_value, start_datetime
        )
        commitments = initialize_combined_commitments(num_devices)
        
        ems_constraints = initialize_df(StorageScheduler.COLUMNS, start, end, resolution)
        ems_constraints["derivative max"] = 15
        ems_constraints["derivative min"] = 0
        
        initial_stocks = soc_at_start
        _, _, results, model = device_scheduler(
            device_constraints, ems_constraints, commitments=commitments, initial_stock=initial_stocks
        )
        
        all_schedules = []
        combined_ems_schedule = [0] * len(market_prices)  

        individual_costs = []
        
        for i in range(num_devices):
            schedule = initialize_series(
                data=[model.ems_power[i, j].value for j in model.j],
                start=start, end=end, resolution=resolution
            )
            all_schedules.append(schedule)
            
            combined_ems_schedule = [combined_ems_schedule[j] + schedule[j] for j in range(len(schedule))]

            costs = sum(schedule[j] * market_prices[j] for j in range(len(market_prices)))
            individual_costs.append((i, costs))
        
        total_costs = sum(cost for _, cost in individual_costs)  

        return all_schedules, combined_ems_schedule, results, individual_costs, total_costs

schedules, combined_ems_schedule, results, individual_costs, total_costs = run_simultaneous_scheduler()

print("\n=== Schedules for All Devices ===")
for i, schedule in enumerate(schedules):
    print(f"\nDevice {i} Schedule:")
    print(f"Time Steps | Power (kW)")
    print("="*30)
    for time, power in zip(range(len(schedule)), schedule): 
        print(f"{start + timedelta(hours=time)} | {power:.2f}")
    
print("\n=== Combined EMS Schedule (All Devices Together) ===")
print(f"Time Steps | Combined Power (kW)")
print("="*30)
for time, power in enumerate(combined_ems_schedule):
    print(f"{start + timedelta(hours=time)} | {power:.2f}")

print("\n=== Individual Costs per Device ===")
for device, cost in individual_costs:
    print(f"Device {device}: {cost:.2f} currency units")

print("\n=== Total Costs ===")
print(f"Total Costs: {total_costs:.2f} currency units")


# Testing Simultaenous Cost Efficiency

## Load and structure 2023 price data

In [ ]:
file_path = "sensor_58.xlsx"
df = pd.read_excel(file_path)
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.date

daily_prices = df.groupby('date')['price'].apply(list)
market_prices_dict = daily_prices.to_dict()

market_prices_24hr = {}

for date, prices in market_prices_dict.items():
    if len(prices) < 24:
        missing_hours = 24 - len(prices)
        prices += [prices[-1]] * missing_hours
        print(f"Date: {date} had missing values. Padded with {missing_hours} additional prices.")
    elif len(prices) > 24:
        prices = prices[:24]
        print(f"Date: {date} had extra values. Truncated to 24 prices.")
    else:
        print(f"Date: {date} has exactly 24 prices.")

    market_prices_24hr[date] = prices

for date, prices in market_prices_24hr.items():
    print(f"Date: {date}, Market Prices: {prices}")

## Simultaneous scheduling simulation with 2023 data: total daily expenses

In [ ]:
resolution = timedelta(hours=1)

# Parameters
soc_at_start = [3, 2, 5, 4] * 4 
soc_max = [11] * 16
soc_min = [0] * 16
target_value = [11] * 16

soc_target_penalty = -100000

daily_total_costs = []
total_unmet_targets = 0

def get_date_range(daily_prices):
    available_dates = sorted(daily_prices.index)
    start_date = pd.to_datetime("2023-01-01").date()
    end_date = pd.to_datetime("2023-01-04").date()    

    selected_dates = [date for date in available_dates if start_date <= date <= end_date]
    if not selected_dates:
        print("No data available for the selected range. Exiting.")
        exit()
    return selected_dates

selected_dates = get_date_range(daily_prices)

for day in selected_dates:
    market_prices = daily_prices[day]
    print(f"\n===== Running Scheduler for {day} =====")

    start = pd.Timestamp(f"{day} 00:00:00")
    end = start + timedelta(hours=24)
    start_datetime = [f"{day} 09:00:00"] * 16
    target_datetime = [f"{day} 13:00:00", f"{day} 17:00:00", f"{day} 14:00:00", f"{day} 16:00:00"] * 4

    def initialize_combined_constraints(num_devices, soc_at_start, soc_max, soc_min, target_datetime, target_value, start_datetime):
        device_constraints = []
        for i in range(num_devices):
            constraints = initialize_df(StorageScheduler.COLUMNS, start, end, resolution)

            start_time = pd.Timestamp(start_datetime[i]) - timedelta(hours=1)
            target_time = pd.Timestamp(target_datetime[i])

            constraints["max"] = soc_max[i] - soc_at_start[i]
            constraints["min"] = soc_min[i] - soc_at_start[i]
            constraints["derivative max"] = 11
            constraints["derivative min"] = 0
            #constraints["min"][target_datetime[i]] = target_value[i] - soc_at_start[i]
            constraints.at[target_time, "min"] = 0 - soc_at_start[i]

            # Disable charging/discharging after the target datetime
            constraints.loc[target_time + resolution:, ["derivative max", "derivative min"]] = 0
            constraints.loc[target_time + resolution:, ["max", "min"]] = constraints.loc[target_time, ["max", "min"]].values

            constraints.loc[:start_time, ["max", "min", "derivative max", "derivative min"]] = 0
            device_constraints.append(constraints)
        return device_constraints

    def initialize_combined_commitments(num_devices):
        commitments = []
        for d in range(num_devices):
            energy_commitment = initialize_df(
                ["quantity", "downwards deviation price", "upwards deviation price", "group"],
                start, end, resolution
            )
            energy_commitment["quantity"] = 0
            energy_commitment["downwards deviation price"] = market_prices
            energy_commitment["upwards deviation price"] = market_prices
            energy_commitment["group"] = list(range(len(energy_commitment)))
            commitments.append(energy_commitment)
                
            stock_commitment = initialize_df(
                ["quantity", "downwards deviation price", "upwards deviation price", "group"],
                start, end, resolution
            )
            stock_commitment["quantity"] = target_value[d] - soc_at_start[i]
            stock_commitment["downwards deviation price"] = soc_target_penalty
            stock_commitment["upwards deviation price"] = soc_target_penalty
            stock_commitment["group"] = list(range(len(stock_commitment)))
            stock_commitment["device"] = d
            stock_commitment["class"] = StockCommitment
            commitments.append(stock_commitment)
        
        return commitments

    def run_simultaneous_scheduler():
        with app.app_context():
            num_devices = len(soc_at_start)
            device_constraints = initialize_combined_constraints(
                num_devices, soc_at_start, soc_max, soc_min, target_datetime, target_value, start_datetime
            )
            commitments = initialize_combined_commitments(num_devices)
            
            ems_constraints = initialize_df(StorageScheduler.COLUMNS, start, end, resolution)
            ems_constraints["derivative max"] = 13.34
            ems_constraints["derivative min"] = 0
            
            initial_stocks = soc_at_start
            _, _, results, model = device_scheduler(
                device_constraints, ems_constraints, commitments=commitments, initial_stock=initial_stocks
            )
            
            print(results.solver.termination_condition)

            all_schedules = []
            combined_ems_schedule = [0] * len(market_prices)  

            individual_costs = []
            unmet_targets = []
            daily_unmet_demand = 0
            
            for i in range(num_devices):
                schedule = initialize_series(
                    data=[model.ems_power[i, j].value for j in model.j],
                    start=start, end=end, resolution=resolution
                )
                all_schedules.append(schedule)
                
                combined_ems_schedule = [combined_ems_schedule[j] + schedule[j] for j in range(len(schedule))]

                costs = sum(schedule[j] * market_prices[j] for j in range(len(market_prices)))
                individual_costs.append((i, costs))

                # Calculate final SoC and check if target is unmet
                final_soc = initial_stocks[i] + sum(schedule)
                if final_soc < target_value[i]:
                    unmet_targets.append((i, final_soc))
                    unmet_demand = target_value[i] - final_soc
                    daily_unmet_demand += unmet_demand
            
            total_costs = sum(cost for _, cost in individual_costs)  

            return all_schedules, combined_ems_schedule, results, individual_costs, total_costs, unmet_targets, daily_unmet_demand

    schedules, combined_ems_schedule, results, individual_costs, total_costs, unmet_targets, daily_unmet_demand = run_simultaneous_scheduler()

    daily_total_costs.append(total_costs)

    
    print("\n=== Schedules for All Devices ===")
    #print(pd.DataFrame(schedules).transpose())
    """
    for i, schedule in enumerate(schedules):
        print(f"\nDevice {i} Schedule:")
        print(f"Time Steps | Power (kW)")
        print("="*30)
        for time, power in zip(range(len(schedule)), schedule): 
            print(f"{start + timedelta(hours=time)} | {power:.2f}")
    """
    
    print("\n=== Combined EMS Schedule (All Devices Together) ===")
    print(f"Time Steps | Combined Power (kW)")
    print("="*30)
    for time, power in enumerate(combined_ems_schedule):
        print(f"{start + timedelta(hours=time)} | {power:.2f}")
    
    """
    print("\n=== Individual Costs per Device ===")
    for device, cost in individual_costs:
        print(f"Device {device}: {cost:.2f} currency units")
    
    print("\n=== Total Costs ===")
    print(f"Total Costs: {total_costs:.2f} currency units")
    """
    if unmet_targets:
        unmet_targets_for_day = len(unmet_targets)
        total_unmet_targets += unmet_targets_for_day
        print("\n=== Devices with Unmet Target SoC ===")
        for device_id, final_soc in unmet_targets:
            unmet_demand = target_value[device_id] - final_soc
            print(f"Device {device_id}: Final SoC = {final_soc:.2f}, Target SoC = {target_value[device_id]}, Unmet Demand = {unmet_demand:.2f}")
        print(f"Daily Unmet Demand: {daily_unmet_demand:.2f}")
    else:
        print("\nAll devices reached their target SoC.")

In [ ]:
def plot_daily_total_costs(daily_total_costs, start_date, num_days):
    dates = pd.date_range(start=start_date, periods=num_days, freq='D')

    plt.figure(figsize=(10, 6))
    plt.plot(dates, daily_total_costs, linestyle='-', color='r')
    plt.title('Total Daily Costs Across All Devices')
    plt.xlabel('Date')
    plt.ylabel('Total Cost')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

%matplotlib inline
plot_daily_total_costs(daily_total_costs, start_date="2023-01-01", num_days=len(selected_dates))

In [ ]:
def print_daily_total_costs(daily_total_costs, selected_dates):
    print("\n=== Daily Total Expenses ===")
    for i, cost in enumerate(daily_total_costs):
        print(f"{selected_dates[i].strftime('%Y-%m-%d')}: {cost:.2f} currency units")

print_daily_total_costs(daily_total_costs, selected_dates)


In [ ]:
def export_daily_costs_to_csv(daily_total_costs, selected_dates, output_file="SimultCostsCase3.csv"):
    data = {
        "Date": [date.strftime('%Y-%m-%d') for date in selected_dates],
        "Total Cost": daily_total_costs
    }
    df = pd.DataFrame(data)

    df.to_csv(output_file, index=False)
    print(f"\nDaily total costs exported successfully to {output_file}")

export_daily_costs_to_csv(daily_total_costs, selected_dates)

# Test for the Simultaneous scheduling runtimes

In [ ]:
import time

start = pd.Timestamp("2020-01-01T00:00:00")
end = pd.Timestamp("2020-01-02T00:00:00")
resolution = timedelta(hours=1)

market_prices = [
    0.8598, 1.4613, 2430.3887, 3000.1779, 18.6619, 369.3274, 169.8719, 174.2279, 174.2279, 174.2279,
    175.4258, 1.9297, 174.2763, 174.2279, 175.2564, 202.6992, 218.4413, 229.9242, 295.1069, 240.7174,
    249.2479, 238.2732, 229.8395, 216.5779
]

def generate_device_data(num_devices):
    soc_at_start = [3, 2, 5, 4] * (num_devices // 4 + 1)
    soc_max = [11] * num_devices
    soc_min = [0] * num_devices
    start_datetime = ["2020-01-01 09:00:00"] * num_devices
    target_datetime = ["2020-01-01 17:00:00", "2020-01-01 17:00:00", "2020-01-01 15:00:00", "2020-01-01 16:00:00"] * (num_devices // 4 + 1)
    target_value = [11] * num_devices
    return soc_at_start[:num_devices], soc_max[:num_devices], soc_min[:num_devices], start_datetime[:num_devices], target_datetime[:num_devices], target_value[:num_devices]

def benchmark_scheduler(device_counts):
    with app.app_context():
        runtimes = []
        for num_devices in device_counts:
            print(f"Running scheduler for {num_devices} devices...")
            
            soc_at_start, soc_max, soc_min, start_datetime, target_datetime, target_value = generate_device_data(num_devices)

            start_time = time.time()

            device_constraints = initialize_combined_constraints(
                num_devices, soc_at_start, soc_max, soc_min, target_datetime, target_value, start_datetime
            )
            commitments = initialize_combined_commitments(num_devices)

            ems_constraints = initialize_df(StorageScheduler.COLUMNS, start, end, resolution)

            per_device_value = 2
            total_derivative_max = num_devices * per_device_value

            # Set EMS constraints
            ems_constraints["derivative max"] = total_derivative_max
            ems_constraints["derivative min"] = 0
            initial_stocks = soc_at_start
            
            _, _, _, _ = device_scheduler(
                device_constraints, ems_constraints, commitments=commitments, initial_stock=initial_stocks
            )
            
            end_time = time.time()
            runtime = end_time - start_time
            runtimes.append(runtime)
            print(f"Completed in {runtime:.2f} seconds.")
        return runtimes

device_counts = [1, 10, 25, 50, 100, 250, 500, 1000]

runtimes = benchmark_scheduler(device_counts)


In [ ]:
%matplotlib inline

plt.figure(figsize=(10, 6))
plt.plot(device_counts, runtimes, marker='o')
plt.title("Computation Time vs Number of Devices")
plt.xlabel("Number of Devices")
plt.ylabel("Computation Time (seconds)")
plt.axhline(y=10, color='red', linestyle='--', label='10-Second Line')
plt.grid()
plt.show()

In [ ]:
filtered_device_counts = [d for d in device_counts if d <= 100]
filtered_runtimes = runtimes[:len(filtered_device_counts)]

plt.figure(figsize=(10, 6))
plt.plot(filtered_device_counts, filtered_runtimes, marker='o', color='royalblue', label='Computation Time')
plt.title("Computation Time vs Number of Devices (up to 100)", fontsize=14, fontweight='bold')
plt.xlabel("Number of Devices", fontsize=12)
plt.ylabel("Computation Time (seconds)", fontsize=12)

plt.axhline(y=10, color='red', linestyle='--', label='10-Second Line')

plt.grid(visible=True, linestyle='--', alpha=0.7)
plt.legend(fontsize=10)
plt.tight_layout()

plt.show()

In [ ]:
runtimes_df = pd.DataFrame({
    'Device Count': device_counts,
    'Runtime (seconds)': runtimes
})

output_file = "SimultTimes.csv"
runtimes_df.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")